In [1]:
import os
nthreads = 1
os.environ["OMP_NUM_THREADS"] = str(nthreads)
os.environ["OPENBLAS_NUM_THREADS"] = str(nthreads)
os.environ["MKL_NUM_THREADS"] = str(nthreads)

In [2]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import inferagni as ia

/home/harrison/miniforge3/envs/proteus/lib/python3.12/site-packages/astropy/config/paths.py:55: AstropyUserWarning: XDG_CONFIG_HOME is set to '/home/harrison/.var/app/com.visualstudio.code/config', but the default location, /home/harrison/.astropy/config, already exists, and takes precedence. This environment variable will be ignored.
  return set_temp_config._get_dir_path(rootname)


In [3]:
# %config InlineBackend.figure_format = 'retina'

In [4]:
datadir = "../src/inferagni/data/"

gr = ia.grid.Grid(datadir)

Loading data from disk...
    Source: /home/harrison/Projects/InferAGNI/src/inferagni/data
Loading grid of scalar quantities
    Grid size: 264600 points
    Input vars:
      Teff            : range [2500      , 5750      ]
      instellation    : range [1         , 1000      ]
      logZ            : range [-2        , 1         ]
      logCO           : range [-3        , 0         ]
      mass_tot        : range [1         , 10        ]
      frac_core       : range [0.2       , 0.7       ]
      frac_atm        : range [0.001     , 0.25      ]
    Output vars: 
      succ, flux_loss, r_bound, p_surf,
      t_surf, r_surf, μ_surf, g_surf, t_phot,
      r_phot, μ_phot, g_phot, vmr_H2,
      vmr_H2O, vmr_CO2, vmr_CO, vmr_O2,
      vmr_OH, vmr_NH3, vmr_NO2, vmr_N2,
      vmr_SO2, vmr_H2S, vmr_H2SO4, Kzz_max,
      conv_ptop, conv_pbot, H_phot
Loading emission spectra
    done
Loading atmosphere profiles
    done
--------------------------------------------------



In [5]:
# Observables (already scaled)

# L 98-59 d
# obs = {
#     "_name":        "L 98-59 d",
#     "r_phot":       [1.521, (0.119,0.098)],
#     "mass_tot":     [2.14,  (0.25,0.29)],
#     "μ_phot":       [9.18,  (2.51,2.41)],
#     "instellation": [5.0,    0.5],
#     "Teff":         [3415.0, 60.0]
# }

obs = ia.planets.get_obs("TOI-421 b")
obs["μ_phot"] = [2.31,(0.03,0.001)]

# Others
# obs = ia.planets.get_obs("K2-18 b")
# obs = ia.planets.get_obs("TRAPPIST-1 c")

Found observations for exoplanet 'TOI-421 b'
    r_phot          :       2.64 ± 0.08      
    mass_tot        :        6.7 ± 0.6       
    Teff            :       5291 ± 64        
    instellation    :    175.878 ± 6.65754   


In [7]:
# Do retrieval
keys, samples = ia.retrieve.run(gr, obs,
                                n_walkers=22, n_procs=14, n_steps=1000,
                                extra_keys=["vmr_H2O","t_surf"])

Prepare interpolators
Interpolator already initialised on frac_core
Interpolator already initialised on r_phot
Interpolator already initialised on mass_tot
Interpolator already initialised on μ_phot
Interpolator already initialised on logZ
Interpolator already initialised on Teff
Interpolator already initialised on t_surf
Interpolator already initialised on logCO
Interpolator already initialised on vmr_H2O
Interpolator already initialised on frac_atm
Interpolator already initialised on instellation
 
Copy grid object into module global scope
Observables:
    r_phot          :       2.64 ± 0.08      
    mass_tot        :        6.7 ± 0.6       
    Teff            :       5291 ± 64        
    instellation    :    175.878 ± 6.65754   
    μ_phot          :       2.31 (+ 0.03       - 0.001     )

Initial guesses for parameters:
    Teff            : linear [   5191.42,    5415.61] w/ 22 walkers
    instellation    :  log10 [  0.694471,          3] w/ 22 walkers
    logZ            : lin

 24%|██▍       | 289/1200 [00:04<00:14, 62.42it/s]Process ForkPoolWorker-16:
Process ForkPoolWorker-21:
Process ForkPoolWorker-27:
Process ForkPoolWorker-24:
Process ForkPoolWorker-25:
Process ForkPoolWorker-23:


emcee: Exception while calling your likelihood function:

Traceback (most recent call last):
Process ForkPoolWorker-22:
Process ForkPoolWorker-26:
Traceback (most recent call last):


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):



KeyboardInterrupt: 

In [ ]:
fig = ia.retrieve.plot_chain(samples,show=True)

In [ ]:
fig = ia.retrieve.plot_corner(keys, samples,show=True)

In [ ]:
# Draw TP profiles from a random posterior sample set and approximate median T(p)

# assemble grid input array in the same order as `gr.input_keys`
X = np.vstack([gr.data.loc[:,k] for k in gr.input_keys]).T

# number of random draws to visualise
rng = np.random.default_rng(42)
max_chain = len(samples)
min_chain = int(max_chain*0.75)
draw_idx = rng.choice(range(min_chain,max_chain,1), size=max_chain-min_chain, replace=True)

profs = []
dists = []
for i in draw_idx:
    # get sample from chain
    params = samples[i,:len(gr.input_keys)]

    # get euclidean distances in parameter-space
    dist_params = np.linalg.norm(X - params, axis=1)

    # get index of closest gridpoint
    idx = int(np.argmin(dist_params))
    t = gr.profs['t'][idx,:].astype(float)
    p = gr.profs['p'][idx,:].astype(float) / ia.util.bar
    r = gr.profs['r'][idx,:].astype(float) / ia.util.R_earth

    dists.append(dist_params[idx])
    profs.append(dict(idx=idx, t=t, p=p, r=r))

print(f"Sampled {len(profs)} profiles")

# stack temperatures and compute median profile
T_stack = np.vstack([pr['t'] for pr in profs])
T_median = np.median(T_stack, axis=0)

# Colormap
sm = mpl.cm.ScalarMappable(cmap='gray',
                           norm=mpl.colors.LogNorm(vmin=np.amin(dists),vmax=np.amax(dists)))

# Plot ensemble and median T(p)
fig, axs = plt.subplots(1,2,figsize=(7,5),sharey=True)
axl,axr = axs
mask = np.argsort(dists)[::-1]
for i in mask:
    pr = profs[i]
    col = sm.to_rgba(dists[i])
    al = 0.5 * (np.amax(dists) - dists[i]) / (np.amax(dists)-np.amin(dists))
    axl.plot(pr['t'], pr['p'], alpha=al, color=col)
    axr.plot(pr['r'], pr['p'], alpha=al, color=col)

for ax in axs:
    ax.grid(True, which='both', alpha=0.3, zorder=-2)
    ax.axhline(20*1e-3, color='orangered')
axr.axvline(obs["r_phot"][0], color='orangered')


axl.set_yscale('log')
axl.invert_yaxis()
axl.set_ylabel('Pressure [bar]')

axl.set_xlabel(r'Temperature [K]')
axr.set_xlabel(r'Radius [R$_\oplus$]')
axr.set_xlim(left=1)


# ax.legend()
fig.tight_layout()
plt.show()